In [2]:
# Các bước chuẩn bị data
import pandas as pd
df_customer=pd.read_csv('03_Customer_Behavior_Data_THPT.csv')
df_item=pd.read_csv('03_Item_information_Data_THPT.csv')
df_shelf=pd.read_csv('03_Shelf_Information_Data_THPT.csv')
df_temp = pd.merge(df_customer, df_item, on=['Shelf ID','Item ID'], how='inner')
df_full = pd.merge(df_temp, df_shelf, on=['Shelf ID'], how='inner')
df_full['Price'] = df_full['Price'].str.replace('₫', '', regex=False)  # Remove the currency symbol
df_full['Price'] = df_full['Price'].str.replace(',', '', regex=False)  # Remove commas
df_full['Price'] = df_full['Price'].astype(float)

In [3]:
#Chắc chắn rằng không có sự trùng lập
print(len(df_full.groupby('Name').filter(lambda x: x['Item ID'].nunique() > 1)))

0


In [4]:
#Kiểm tra có bao nhiêu tên cột
print(df_full.columns)

Index(['Shelf ID', 'Person ID', 'Date', 'Time', 'Day', 'Age', 'Gender',
       'Height_x', 'Weight', 'Married status', 'Item ID',
       'Looking at item (s)', 'Holding the item (s)', 'Holding the bag',
       'Picking up item', 'Returning item', 'Putting item into bag',
       'Taking item out of bag', 'Putting item into bag in the 2nd time',
       'Name', 'Origin', 'Price', 'Marketing strategy', 'Discount ', 'Rate',
       'Description', 'Width', 'Height_y', 'Length', 'Number of items',
       'Shape'],
      dtype='object')


In [5]:
#câu hỏi 1: Thống kê 5 mặt hàng với lượng khách hàng nhìn thấy nhiều nhất

group_looked = df_full.groupby('Name')['Looking at item (s)'].sum().reset_index(name='Tổng lượt nhìn')
top_5_items_looked = group_looked.sort_values(by='Tổng lượt nhìn', ascending=False).head(5)
print(top_5_items_looked)

                     Name  Tổng lượt nhìn
108   Sữa chua uống Probi           11503
109  Sữa chua uống Yakult           11454
110           Sữa ông thọ            7712
4           Bim bim Oishi            7415
77   Snack khoai tây Lays            7051


In [6]:
#câu hỏi 2: Thống kê 5 mặt hàng có tổng thời gian nhìn và cầm xem lâu nhất
df_full['total time (s)']=df_full['Looking at item (s)'] + df_full['Holding the item (s)']
grouped = df_full.groupby('Name')['total time (s)'].sum().reset_index(name='total (s)')
top_5_items = grouped.sort_values(by='total (s)', ascending=False).head(5)
print(top_5_items)

                     Name  total (s)
109  Sữa chua uống Yakult      22896
108   Sữa chua uống Probi      22896
110           Sữa ông thọ      13939
4           Bim bim Oishi      13866
77   Snack khoai tây Lays      13362


In [7]:
#câu hỏi 3: Thống kê 5 mặt hàng được mua nhiều nhất

df_full_buy = df_full[(df_full['Putting item into bag']==True)&(df_full['Taking item out of bag']==False)]
df_full_buy.groupby(['Name']).size().sort_values(ascending=False).head(5)

Name
Lốc 4 hộp sữa tươi ít đường TH true MILK    159
Kem tràng tiền                              159
Sữa bột Milo                                158
Bánh trứng Custard                          156
Dầu gội Romano                               99
dtype: int64

In [8]:
#câu hỏi 4: Thống kê 5 mặt hàng thường được cầm lên rồi trả lại nhiều nhất
df_full_filter_q4 = df_full[(df_full['Picking up item'] == True) & (df_full['Returning item'] == True)]
df_full_filter_q4.groupby('Name').size().sort_values(ascending=False).head(5)

Name
4 hộp sữa lúa mạch Milo 180ml    134
Snack khoai tây Lays             127
Mý ý SG Food                     117
Nước lẩu Barona                  116
Bánh Chocopie                    114
dtype: int64

In [13]:
#câu hỏi 5: Các nhóm khách hàng theo độ tuổi (Thiếu niên: 18 - 30; Trung niên: 31 - 60; Cao tuổi: > 60) mua mặt hàng nào nhiều nhất?

In [14]:
#thiếu niên
df_buy_filter_thieunien = df_full_buy[(df_full_buy['Age'] >= 18) & (df_full_buy['Age'] <= 30)].copy()
print('Thiếu niên:\n',df_buy_filter_thieunien.groupby('Name').size().sort_values(ascending=False).head(1))

Thiếu niên:
 Name
Bánh trứng Custard    42
dtype: int64


In [15]:
#trung niên
df_buy_filter_trungnien = df_full_buy[(df_full_buy['Age']>=31) & (df_full_buy['Age']<=60)].copy()
print('Trung niên:\n',df_buy_filter_trungnien.groupby('Name').size().sort_values(ascending=False).head(1))

Trung niên:
 Name
Kem tràng tiền    89
dtype: int64


In [16]:
#cao tuổi
df_buy_filter_caotuoi = df_full_buy[df_full_buy['Age']>60].copy()
print('Cao tuổi:\n',df_buy_filter_caotuoi.groupby('Name').size().sort_values(ascending=False).head(1))

Cao tuổi:
 Name
Sữa bột Milo    49
dtype: int64


In [16]:
#câu hỏi 6: Ngày nào trong tuần có doanh thu cao nhất?

In [18]:
df_full_buy.groupby('Day')['Price'].sum().sort_values(ascending=False)

Day
Saturday     192021600.0
Sunday       184134400.0
Tuesday       55067600.0
Thursday      51941300.0
Friday        50958000.0
Wednesday     42968200.0
Monday        39364600.0
Name: Price, dtype: float64

In [17]:
#câu hỏi 7: Trong 3 nhóm tuổi sau: Thiếu niên (18 - 30),  Trung niên (31 - 60), Cao tuổi: (> 60),  nhóm tuổi nào có số người đi siêu thị nhiều nhất?


df_full_filter_thieunien = df_full[(df_full['Age'] >= 18) & (df_full['Age'] <= 30)].copy()
df_full_filter_trungnien = df_full[(df_full['Age']>=31) & (df_full['Age']<=60)].copy()
df_full_filter_caotuoi = df_full[df_full['Age']>60].copy()

print(f"So nguoi di sieu thi cua nhom thieu nien la {df_full_filter_thieunien['Person ID'].nunique()}, trung nien la {df_full_filter_trungnien['Person ID'].nunique()}, cao tuoi la {df_full_filter_caotuoi['Person ID'].nunique()}")


So nguoi di sieu thi cua nhom thieu nien la 339, trung nien la 753, cao tuoi la 408


In [18]:
#câu hỏi 8: Top 5 các mặt hàng giảm giá được người dùng mua nhiều nhất?
df_full_buy[df_full_buy['Discount ']>0].groupby('Name').size().sort_values(ascending=False).head(5)

Name
Kem tràng tiền        159
Sữa bột Milo          158
Bánh trứng Custard    156
Dầu gội Romano         99
Khăn tắm Shine         94
dtype: int64

In [19]:
#câu hỏi 9: Top 3 quầy hàng nào có lượt cầm xem nhiều nhất?
group_holding = df_full.groupby('Shelf ID')['Holding the item (s)'].sum().reset_index(name='Tổng lượt cầm xem')
top_3_sections_holding = group_holding.sort_values(by='Tổng lượt cầm xem', ascending=False).head(3)
print(top_3_sections_holding)

   Shelf ID  Tổng lượt cầm xem
7         7             100194
0         0              74955
1         1              58668


In [20]:
#câu hỏi 10: Trong số các sản phẩm được đánh giá cao nhất, top 3 sản phẩm nào được mua nhiều nhất?
df_full_buy[(df_full_buy['Rate']==5)].groupby('Name').size().sort_values(ascending=False).head(3)

Name
Kem tràng tiền    159
Khăn mặt Shine     94
Khăn tắm Shine     94
dtype: int64

In [21]:
#câu hỏi 11: Câu hỏi mở: Tìm ra mặt hàng hay được chú ý nhất của những nhóm khách hàng cụ thể

Để trả lời câu hỏi trên ta sẽ phân tích bằng cách trả lời sau đây

Thế nào là 1 mặt hàng được chú ý ?

Để giải quyết em đưa ra cách tính thời gian chú ý như sau

Thoi gian chu y = Thoi gian looking + thoi gian holding (Khi chu y 1 san pham nguoi dung se co 2 thao tac hoac la nhin no hoac la cam nam no)

Từ đó để tìm sản phẩm được chú ý em đưa ra tiêu chí sau 

San pham chu y trong 1 giao dịch : Thoi gian chu y > 30s hoac trong giao dich do san pham do duoc 1 nguoi nao do pick up len (nho giai thich vi sao tinh chat nay goi la chu y)

Khi đó ta sẽ thống kê những sản phẩm nào thoả mãn tiêu chí trên nhiều lần sẽ được xếp hạng và lọc ra cho từng lứa tuổi khác nhau


In [22]:
cond_thieunien = (df_full_filter_thieunien['Looking at item (s)'] + df_full_filter_thieunien['Holding the item (s)']  > 30) | (df_full_filter_thieunien['Picking up item']==True)

cond_trungnien = (df_full_filter_trungnien['Looking at item (s)'] + df_full_filter_trungnien['Holding the item (s)']  > 30) | (df_full_filter_trungnien['Picking up item']==True)

cond_caotuoi = (df_full_filter_caotuoi['Looking at item (s)'] + df_full_filter_caotuoi['Holding the item (s)']  > 30) | (df_full_filter_caotuoi['Picking up item']==True)


print(f" Top mat hang thieu nien la {df_full_filter_thieunien[cond_thieunien].groupby('Name').size().sort_values(ascending=False).head(10).index.tolist()}")
print(f" Top mat hang trung nien la {df_full_filter_trungnien[cond_trungnien].groupby('Name').size().sort_values(ascending=False).head(10).index.tolist()}")
print(f" Top mat hang cao tuoi la {df_full_filter_caotuoi[cond_caotuoi].groupby('Name').size().sort_values(ascending=False).head(10).index.tolist()}")

 Top mat hang thieu nien la ['Sữa bột Anmum Materna', 'Bánh Chocopie', 'Snack khoai tây Lays', 'Kẹo cao su Cool Air Fresh', 'Bim bim Oishi', 'Bánh trứng Custard', '4 hộp sữa lúa mạch Milo 180ml', 'Lốc 4 hộp sữa tươi ít đường TH true MILK', 'Nước lẩu Barona', 'Viên thả lẩu cầu tre']
 Top mat hang trung nien la ['Snack khoai tây Lays', 'Sữa ông thọ', 'Thạch rau câu Đức Hạnh', 'Kem tràng tiền', '4 hộp sữa lúa mạch Milo 180ml', 'Sữa chua uống Probi', 'Sữa chua Vinamik', 'Thanh cua Akira', 'Hạt hướng dương Chacheer', 'Sữa bột Ensure Gold lúa mạch']
 Top mat hang cao tuoi la ['Bánh trứng Custard', 'Xúc xích CP', 'Kẹo dẻo Alphelibe', 'Sữa bột Milo', 'Bibigo Mandu', 'Bim bim Oishi', 'Hạt hướng dương Chacheer', 'Sửa chua TH', 'Sữa chua Susu', 'Bánh Chocopie']


In [23]:
#câu hỏi 12: Câu hỏi mở: Có những mặt hàng nào cần được sắp xếp lại trong cửa hàng không? Tại sao?

Để trả lời câu hỏi trên ta sẽ phân tích bằng cách trả lời 2 câu sau đây

- Thế nào là mặt hàng cần được sắp xếp lại ?

    - Trong ngữ cảnh này, cần sắp xếp lại ko có nghĩa là món hàng đó lộn xộn

    - Hiểu đúng hơn là cần sắp xếp vị trí để thu hút đc nhiều người hơn 

- Tiêu chuẩn nào để chọn ra được mặt hàng được sắp xếp lại ?

Dựa vào câu trả lời trên ta có 2 tiêu chuẩn sau 

    - Tìm nx mặt hàng mà có thời gian chú ý thấp (lấy top 10 thap nhat của tổng thời gian chú ý của tất cả các mặt hàng )


    - Tim nx mat hang không hoặc ít được mua (lấy top10 ) (df_buy)

Dựa vào 2 tiêu chí đó cách làm cụ thể là:

- Những mặt hàng mà có thời gian chú ý thấp: Em lấy top 10 những sản phẩm có tổng thời gian nhìn và cầm ít nhất và quantile 0.05 những sản phẩm có tổng thời gian nhìn và cầm ít nhất, sau khi dùng python kết quả là : (Ghi ra)

- Ngoài ra những mặt hàng ít người mua bao gồm (Ghi ra). Đây là những mặt hàng theo em cần cân nhắc loại khỏi cửa hàng để giảm chi phí hoặc để nó cùng với mặt hàng cùng loại đang bán chạy khác
    


Tìm top 10 sản phẩm có thời lượng chú ý ít nhất

In [24]:
#thời gian chú ý được xác định bằng tổng thời gian người đó cầm sản phẩm đó lên và nhìn vào sản phẩm đó
# nếu chỉ xác đinh thời gian chú ý bằng thời gian người đó xem sẽ không thể bao quát được trường hợp nếu người đó lấy sản phẩm mà không nhìn (bằng thói quen)
thoi_gian_chu_y=df_full_filter_thieunien['Looking at item (s)'] + df_full_filter_thieunien['Holding the item (s)']
print('top 10 sản phẩm có thời lượng chú ý ít nhất:')
thoi_gian_chu_y.groupby(df_full['Name']).sum().sort_values(ascending=True).head(10)

top 10 sản phẩm có thời lượng chú ý ít nhất:


Name
Sách cho trẻ từ 1-2 tuổi                                                82
Truyện tranh Conan                                                     132
Sách cho trẻ sơ sinh đến 6 tháng tuổi                                  255
Nui Safoco                                                             265
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)    272
Strongbow dâu đen                                                      279
Gấu bông Miyu                                                          285
Sách Chào con Ba mẹ đã sẵn sàng                                        307
Fanta                                                                  352
Máy xay sinh tố Comet                                                  356
dtype: int64

Tìm các sản phẩm có thời lượng chú ý ít hơn hoặc bằng phân vị 5%

In [25]:
# Tính tổng thời gian chú ý theo sản phẩm
tong_thoi_gian_chu_y_theo_san_pham = thoi_gian_chu_y.groupby(df_full['Name']).sum()

# Tìm giá trị tại phân vị 5% (quantile 0.05)
nguong_5_percent = tong_thoi_gian_chu_y_theo_san_pham.quantile(0.05)

# Lọc ra các sản phẩm có thời lượng chú ý nằm dưới mức 5%
san_pham_duoi_nguong_5_percent = tong_thoi_gian_chu_y_theo_san_pham[tong_thoi_gian_chu_y_theo_san_pham <= nguong_5_percent]

# In ra danh sách các sản phẩm này
print('Các sản phẩm có thời lượng chú ý ít hơn hoặc bằng phân vị 5%:')
print(san_pham_duoi_nguong_5_percent.sort_values(ascending=False))

Các sản phẩm có thời lượng chú ý ít hơn hoặc bằng phân vị 5%:
Name
Gấu bông Miyu                                                          285
Strongbow dâu đen                                                      279
Tã dán sơ sinh Bobby siêu mỏng Newborn 70 miếng (cho bé dưới 5kg)    272
Nui Safoco                                                             265
Sách cho trẻ sơ sinh đến 6 tháng tuổi                                  255
Truyện tranh Conan                                                     132
Sách cho trẻ từ 1-2 tuổi                                                82
dtype: int64


Nhận thấy không có sản phẩm nào không được mua


In [26]:
#Những sản phẩm không được mua được xác định bằng số sản phẩm có trong df_full.Name mà không có trong df_full_buy['Name'] 
df_full_not_buy = df_full[~df_full['Name'].isin(df_full_buy['Name'])]
san_pham_khong_duoc_mua = df_full_not_buy['Name'].unique()
print('Số lượng sản phẩm không được mua:', san_pham_khong_duoc_mua)

Số lượng sản phẩm không được mua: []


Sản phẩm được mua ít nhất là Ðèn bàn Ðiện Quang với 2 lần

In [27]:
#In số lượt mua của từng sản phẩm theo thứ tự từ bé đến lớn
df_full_buy.groupby('Name').size().sort_values(ascending=True).head(10)

Name
Ðèn bàn Ðiện Quang                       2
Nồi cơm điện Kangaroo                    4
Phích cắm điện Ðiện Quang                4
Sách cho trẻ từ 2-3 tuổi                 4
Sách cho trẻ từ 4-5 tuổi                 5
Sách Thám tử Sherlock Home               5
Ðậu đỏ                                   5
Sách Ðạo Tình                            6
Sách cho trẻ sơ sinh đến 6 tháng tuổi    6
Bếp điện từ Sunhouse                     7
dtype: int64